In [1]:
import cv2

In [2]:
import numpy as np
import pandas as pd
import os
import pickle

from keras.preprocessing import image
from sklearn.metrics import accuracy_score

C:\Users\hp\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\hp\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\hp\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\hp\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:52

In [3]:
def variance_of_laplacian(image):
	return cv2.Laplacian(image, cv2.CV_64F).var()





X_test = []
y_test = []
y_pred = []
val_blur = []
val_nblur = []
threshold = 215
input_size = (192, 192)


In [4]:
dgbset = pd.read_excel('C:/Users/hp/Desktop/CERTH_ImageBlurDataset/EvaluationSet/DigitalBlurSet.xlsx')
nbset = pd.read_excel('C:/Users/hp/Desktop/CERTH_ImageBlurDataset/EvaluationSet/NaturalBlurSet.xlsx')


In [5]:
dgbset['MyDigital Blur'] = dgbset['MyDigital Blur'].apply(lambda x : x.strip())
dgbset = dgbset.rename(index=str, columns={"Unnamed: 1": "Blur Label"})
nbset['Image Name'] = nbset['Image Name'].apply(lambda x : x.strip())

In [6]:
folderpath = 'C:/Users/hp/Desktop/CERTH_ImageBlurDataset/EvaluationSet/DigitalBlurSet/'

# load image arrays
for filename in os.listdir(folderpath):
    if filename != '.DS_Store':
        imagepath = folderpath + filename
        img = image.load_img(imagepath, target_size=input_size)
        X_test.append(np.asarray(img))
        blur = dgbset[dgbset['MyDigital Blur'] == filename].iloc[0]['Blur Label']
        gray = cv2.cvtColor(np.asarray(img), cv2.COLOR_BGR2GRAY)
        maxval = np.max(cv2.convertScaleAbs(cv2.Laplacian(gray,3)))
        if maxval < threshold:
            y_pred.append(1)
        else:
            y_pred.append(0)

        if blur == 1:
            y_test.append(1)
            val_blur.append(maxval)
        else:
            y_test.append(0)
            val_nblur.append(maxval)
    else:
        print(filename, 'not a pic')

print("Digitally Blurred Evaluated...")


Digitally Blurred Evaluated...


In [7]:
folderpath = 'C:/Users/hp/Desktop/CERTH_ImageBlurDataset/EvaluationSet/NaturalBlurSet/'

# load image arrays
for filename in os.listdir(folderpath):
    if filename != '.DS_Store':
        imagepath = folderpath + filename
        img = image.load_img(imagepath, target_size=input_size)
        X_test.append(np.asarray(img))
        blur = nbset[nbset['Image Name'] == filename.split('.')[0]].iloc[0]['Blur Label']
        gray = cv2.cvtColor(np.asarray(img), cv2.COLOR_BGR2GRAY)
        maxval = np.max(cv2.convertScaleAbs(cv2.Laplacian(gray,3)))
        if maxval < threshold:
            y_pred.append(1)
        else:
            y_pred.append(0)
        if blur == 1:
            y_test.append(1)
            val_blur.append(maxval)
        else:
            y_test.append(0)
            val_nblur.append(maxval)
    else:
        print(filename, 'not a pic')

print("Naturally Blurred Evaluated...")

Naturally Blurred Evaluated...


In [8]:
accuracy = accuracy_score(y_test, y_pred)
print("accuracy: {:.2f}%".format(accuracy * 100))


accuracy: 63.58%


In [9]:
from scipy import stats
print(np.percentile(val_blur,60))
print(np.percentile(val_nblur,15))

print(stats.describe(val_blur))
print(stats.describe(val_nblur))


255.0
255.0
DescribeResult(nobs=861, minmax=(17, 255), mean=201.98606271777004, variance=5369.678875293736, skewness=-1.0729189514344002, kurtosis=-0.3269821207133945)
DescribeResult(nobs=619, minmax=(188, 255), mean=254.8675282714055, variance=7.610254560283579, skewness=-23.316063879349564, kurtosis=557.8368200149888)
